# QUESTIONS


1. Configure an identity provider
- Configure your OpenShift cluster to use an HTPasswd identity provider with the following
- The name of the identity provider is: ex280-htpasswd
- The name of the secret is: ex280-idp-secret
- The user account armstrong is present and can log in with password indionce
- The user account collins is present and can log in with password veraster
- The user account aldrin is present and can log in with password roonkere The user
- account jobs is present and can log in with password sestiver
- The user account wozniak is present and can log in with password glegunge

In [2]:
$ sudo yum install httpd-tools -y

$ htpasswd -c -b -B <file-name> <user-name> <password>      // -c -- new-file creation 

# Now add  user accounts mentioned w.r.t their password.
$ htpasswd -b -B openshift-passwd collins veraster 
$ htpasswd -b -B openshift-passwd aldrin roonkere 
$ htpasswd -b -B openshift-passwd jobs sestiver
$ htpasswd -b -B openshift-passwd wozniak glegunge

# Check whether the file has been created with menntioned user-accounts  
$ ls -ltrh
$ cat openshift-passwd

# Create secret using the file openshift-passwd 
$ oc create secret generic ex280-secret -n openshift-config --from-file htpasswd=openshift-passwd

# Check the create secret in specified namespace
$ oc get secrets -n openshift-config

# now add the secret in Cluster Oauth
$ oc get oauth -o yaml > oauth.yaml
###################################################

apiVersion: v1
items:
- apiVersion: config.openshift.io/v1
  kind: OAuth
  metadata:
    annotations:
      include.release.openshift.io/ibm-cloud-managed: "true"
      include.release.openshift.io/self-managed-high-availability: "true"
      include.release.openshift.io/single-node-developer: "true"
      release.openshift.io/create-only: "true"
    creationTimestamp: "2024-12-30T16:30:46Z"
    generation: 2
    name: cluster
    ownerReferences:
    - apiVersion: config.openshift.io/v1
      kind: ClusterVersion
      name: version
      uid: a6e67508-ef84-4654-965a-fd690c2671b3
    resourceVersion: "1251710"
    uid: abfb432c-4325-4f09-b0b4-851eb4e26f9d
  spec:
    identityProviders:
    - htpasswd:
        fileData:
          name: ex280-secret  // add the created secret here and save the file.
      mappingMethod: claim
      name: htpasswd_provider
      type: HTPasswd
kind: List
metadata:
  resourceVersion: ""


###################################################

$ vi oauth.yaml

$ oc apply -f oauth.yaml

# Check if you can login using the mentioned users.




SyntaxError: invalid syntax (566725104.py, line 1)

Question-2:
- Configure cluster permissions
- Configure your OpenShift cluster to meet the following requirements:
- The user account jobs can perform cluster administration tasks
- The user account wozniak can create projects
- The user account wozniak cannot perform cluster administration tasks The user account armstrong cannot create projects
- The user account kubeadmin is not present

In [ ]:
oc get clusterrole | grep cluster-admin

$ oc adm policy add-role-to-user cluster-admin jobs

$ oc adm policy add-role-to-user self-provsioner wozniak

$ oc delete secret kubeadmin -n kube-system 


Question-3:
- configure project permissions. Configure your OpenShift cluster to meet the
- following requirements: The following projects exist:
- apollo
- manhattan
- gemini
- bluebook
- titan
- The user account armstrong is an administrator for project apollo and project gemini The user account wozniak can view project titan but not
administer or delete it.

In [ ]:
# Create projects mentioned
$ oc new-project apollo
$ oc new-project manhattan
$ oc new-project gemini
$ oc new-project bluebook
$ oc new-project titan

#admin fro project apollo for use armstrong
$ oc project apollo
$ oc adm policy add-role-to-user admin armstrong -n apollo

#USer wozniak can only view project titan but not admin or delete it
$ oc adm policy add-role-to-user view wozniak -n titan


Question-4:
- Configure groups
- Configure your OpenShift cluster to meet the following requirements:
- The user account armstrong is a member of the commander group
- The user account collins is a member of the pilot group
- The user account aldrin is a member of the pilot group
- Members of the commander group have edit permission in the apollo proj
- Members of the pilot group have view permission in the apollo project


In [ ]:
$ oc adm groups new commander
$ oc adm groups new pilot
$ oc adm groups add-users commander armstrong
$ oc adm groups add-users pilot collins,aldrin

$ oc adm policy add-role-to-group edit commander -n apollo
$ oc adm policy add-role-to-group view pilot -n apollo



Question-5:
- Configure quotas
- Configure your OpenShift cluster to use quotas in the manhattan project with the following require
- The name of the quota is: ex280-quota
- The amount of memory consumed across all containers may not exceed 161
- The total amount of CPU consumed across all containers may not exceed 2 full cores
- The maximum number of replication controllers does not exceed 3
- The maximum number of pods does not exceed 3
- The maximum number of services does not exceed 6


In [ ]:
$ oc project manhattan
$ oc create quota ex280-quota --hard=cpu=2,services=6,pods=3,replicationControllers=3,memory=161Mi --dry-run -o yaml > quota.yaml

#############################################
apiVersion: v1
kind: ResourceQuota
metadata:
  creationTimestamp: null
  name: ex280-quota
spec:
  hard:
    limits.cpu: "2"                 #add limits
    limits.memory: 161Mi            #add limits
    pods: "3"
    replicationControllers: "3"
    services: "6"
status: {}
#############################################  

$ oc create -f qouta.yaml 


Question-6:
- Scale an application manually
- Ensure that there are exactly 5 replicas of the minion application in the gru project.

In [ ]:
$ oc scale deploy <deploy-name> --replicas=6 -n <namespace> 

Question-7:
- Scale an application automatically
- Automatically scale the hydra deployment in the lerna project with the follow
- Minimum number of pods: 2
- Maximum number of pods: 9
- Target average CPU utilization per pod: 60 percent
- The pods require 25m CPU time to operate
- The pods must not consume more than 180 CPU time


In [ ]:
$ oc autoscale deploy hydra --min=2 --max=9 --cpu-percent=60
$ oc set resource deployment hydra --requests 25m --limits 180m

Question-8:
- Configure a secure route
- Configure the oxcart application in the area51 project with the following
- requirements:
- The application uses a secure route called oxcart
- Traffic between the client and the router is encrypted
- Traffic between the router and the service is unencrypted
- The route uses a CA signed certificate with the following subject fields:
- /C=US/ST=NV/L=Hiko/O=CIA/OU=USAF/CN=classified.apps.domain20.example.com
- The application is reachable only at the following address:
- https://classified.apps.domain28.example.com
- The application produces output

In [ ]:
$ openssl req x509 -sha256 -nodes -days 365 -newkey rsa:4096 -keyout private.key -out certificate.crt



Question-9:
- Configure application data
- Deploy an application using the openshift/hello-openshift image that meets the following requirements;
- The application is part of a project named: acid
- The application is named: phosphoric
- The application uses a key named RESPONSE in a configuration map named sedicen
- The application is running and available at http://phosphoric
- acid.apps.domain20.example.com and displ
- Soda pop won't stop can't stop



In [3]:
$ oc new-app phosphoric -i openshift/hello-openshift -n acid 
$ oc expose svc phosphoric --hostname=http://phosphoric.acid.apps.domain20.example.com
$ oc create configmap sedicen --from-literal REPSONSE="Soda pop won't stop can't stop"
$ oc set env deploy phosphoric --from=cm/sedicen


SyntaxError: invalid syntax (2981516855.py, line 1)

Question-10:
- Deploy an application
- EX280 exam prepared for
- Deploy the chart named redhat-movie in the project ascii-movie from the repository http://helm.domain20.example.com/chart//
- You may use the telnet or ne commands to validate the deployment.


In [4]:
$ oc new-project ascii-movie
$ helm repo add exam http://helm.domain20.example.com/chart//
$ helm search repo exam
$ helm install redhat-movie exam/redhat-movie -n ascii-movie
$ oc get all -n ascii-movie  
$ oc get route 
$ telnet <route-url>

SyntaxError: invalid syntax (2836486986.py, line 1)

Queston-11:
- Configure a secret
- Configure a secret in the math project with the following requirements:
- The name of the secret is: magic
- The secret defines a key with name: decoder_ring
- 000 VRCKukQdrosbcl0c2ZYhDk- The secret defines the key with value:XpWy9KdcP3T


In [5]:
$ oc new-project Math
$ oc create secret generic magic --from-literal decoder_ring=XpWy9KdcP3T
$ oc get secrets


SyntaxError: invalid syntax (764961077.py, line 1)

Question-12:
- Configure an application to use a secret
- Configure the application called qed in the math project with the following
- requirements:
- The application uses the secret previously created called: magic The secret defines an environment variable with name: DECODER_RING
- The application output no longer displays: Sorry, application is not configured correctly.


In [6]:
$ oc set env deploy qed --from=secret/magic
$ oc get deploy qed -o yaml
# check env in spec section whether the secret was injected or not.

SyntaxError: invalid syntax (3846395621.py, line 1)

Queston-13:
- Configure a service account
- Configure a service account in the apples project to meet the following
- requirements:
- The name of the service account is ex280sa
- The service account allows pods to be run as any available user


In [ ]:
$ oc create sa ex280sa -n apples 
$ oc get scc 
$ oc adm policy add-scc-to-user anyuid -z ex280a
$ oc auth can-i '*' '*'



Question-14:
- Deploy an application
- Deploy the application called oranges in the apples project so that the following
- condition
- No configuration components have been added or removed
- The application produces output
- The application uses the ex280sa service account


In [7]:
$ oc set sa deploy/deploy-name ex280sa
$ curl route-name
$ oc get endpoints
$ oc get po --show-labels
$ oc edit svc <service-name>
# add correct label inside service
$ oc get svc,endpoints



SyntaxError: invalid syntax (2986838067.py, line 1)

Question-15:
- Deploy an application
- Deploy the application called atlas in the mercury project so that the following condition
- No configuration components have been added or removed
- The application produces output
    

In [ ]:
$ oc project mercury
$ oc get deploy atlas
$ oc edit deploy atlas
#change requests: 80 Gi or remove the resources field


Question-16:

- Configure a network policy using the database and checker projects with the following requirements:
- The database project has network policy with the name db-allow-sysql-conn based on pod selector label
- network.openshift.io/policy-group
- Connections to the database project are restricted to deployments from the
- checker project The network policy is filtered by project selector using the team
- devsecops label and pod selector using the deploymentb-sysql label
- The application can establish a connection to port 3306/TCP
- You can check your work by examining the logs in the checker project.